## Visualization of model performance

In [ ]:
import pathlib
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split

from models import ConvRegv2
from mlcore.training import make_predictions

#### Model and Data Loading

In [ ]:
# Load model
MODEL_DIR = pathlib.Path().cwd() / 'trained_models'
MODEL_FNAME = 'cnn_reg_1690938188.pt'
RANDOM_SEED = 42
TEST_RATIO = 0.2
BATCH_SIZE = 32
WINDOW_SIZE = 1000

model = ConvRegv2(2, BATCH_SIZE)
model.load_state_dict(torch.load(MODEL_DIR / MODEL_FNAME))

In [ ]:
# Load data and build Torch datasets
X, y = [], []

# Load the dataset
data_dir = '../../../data/pulses/single_pulse/'
pulse_list = np.load(data_dir + f'vp_single_num1000_win{WINDOW_SIZE}_pad60.npz')
pulses = list(pulse_list['pulses'])
print(len(pulses))

X, y = [], []
random.shuffle(pulses)
for element in pulses:
    X.append(element[0:2,:]) # I and Q timestreams
    arr_time = np.argwhere(element[2] == 1).item()
    pulse_height = element[3][arr_time].item()
    y.append(np.array([arr_time / WINDOW_SIZE, pulse_height]).reshape(1,2)) # Scaling arrival time [0, WINDOW_SIZE] -> [0, 1]


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_RATIO, # Ratio of test data to use from full dataset; Training is the complement
    random_state=RANDOM_SEED
)

# Convert the lists to Tensors. Converting to np arrays first based on warning from torch
X_train = torch.Tensor(np.array(X_train))
X_test = torch.Tensor(np.array(X_test))
y_train = torch.Tensor(np.array(y_train))
y_test = torch.Tensor(np.array(y_test))

# Convert from numpy arrays to Tensors and create datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

### Model Predictions and Visualization

In [ ]:
# Pick k random samples/labels from the test data and plot them along with the predictions
test_samples = []
test_labels = []

for sample, label in random.sample(list(test_dataset), k=len(test_dataset)): # random.sample samples k elements from the given population without replacement; returns list of samples.
    test_samples.append(sample)
    test_labels.append(label)

print(f'Test Sample Shape: {test_samples[0].shape}, Test Label Shape: {test_labels[0].shape}')
preds = make_predictions(model, [x.unsqueeze(dim=0) for x in test_samples]) # returns a tensor
print(f'Preds shape {preds[0].shape}')

In [ ]:
# Since the train/test loss was so low and the absolute error was low, lets determine the difference between the predicted and
# target arival times and plot this
# Note the multiplication by 1000 to get back to the arrival time element
arrival_diff = [torch.abs(WINDOW_SIZE * y_pred[0][0][0] - WINDOW_SIZE * y_true[0][0]).item() for y_pred, y_true in zip(preds, test_labels)]

plt.figure()
plt.plot(np.arange(len(arrival_diff)), arrival_diff)
plt.xlabel('Test Sample')
plt.ylabel('Error in Photon Arrival (us)')
plt.show()

In [ ]:
# Now that the error is known, let's plot the actual values for both the predictions and the targets
plt.figure(figsize=(10,5))
#plt.plot(np.arange(len(preds)), [WINDOW_SIZE*pred[0][0][0].item() for pred in preds], label='Predicted')
#plt.plot(np.arange(len(test_labels)), [WINDOW_SIZE*label[0][0].item() for label in test_labels], label='Target')
plt.scatter(np.arange(len(preds)), [WINDOW_SIZE*pred[0][0][0].item() for pred in preds], marker='+', label='Predicted')
plt.scatter(np.arange(len(test_labels)), [WINDOW_SIZE*label[0][0].item() for label in test_labels], label='Target')
plt.xlabel('Test Sample')
plt.ylabel('Photon Arrival (us)')
plt.legend()
plt.show()

In [ ]:
# Now lets do the same with the pulse height
height_diff = [torch.abs(y_pred[0][0][1] - y_true[0][1]).item() for y_pred, y_true in zip(preds, test_labels)]

plt.figure()
plt.plot(np.arange(len(height_diff)), height_diff)
plt.xlabel('Test Sample')
plt.ylabel('Error in Pulse Height (a.u.)')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
#plt.plot(np.arange(len(preds)), [pred[0][0][1].item() for pred in preds], label='Predicted')
#plt.plot(np.arange(len(test_labels)), [label[0][1].item() for label in test_labels], label='Target')
plt.scatter(np.arange(len(preds)), [pred[0][0][1].item() for pred in preds], label='Predicted')
plt.scatter(np.arange(len(test_labels)), [label[0][1].item() for label in test_labels], marker='+', label='Target')
plt.xlabel('Test Sample')
plt.ylabel('Pulse Height (a.u.)')
plt.legend()
plt.show()